#### Create access token

In [ ]:
import pandas as pd
import webbrowser
# Import the required module from the fyers_apiv3 package
from fyers_apiv3 import fyersModel
from datetime import datetime, timedelta
import pandas as pd
import time

# Configuration

In [ ]:
def generate_fyers_access_token(
    client_id: str,
    secret_key: str,
    redirect_uri: str,
    state: str = "sample_state"
) -> dict:
    """
    Handles FYERS OAuth flow and returns access token response.

    Returns
    -------
    dict
        Full response containing access_token, refresh_token, expiry, etc.
    """

    # -------------------------------
    # Step 1: Generate auth URL
    # -------------------------------
    session = fyersModel.SessionModel(
        client_id=client_id,
        secret_key=secret_key,
        redirect_uri=redirect_uri,
        response_type="code",
        state=state
    )

    auth_url = session.generate_authcode()
    webbrowser.open(auth_url)

    # -------------------------------
    # Step 2: User pastes redirected URL
    # -------------------------------
    raw_code = input("Paste redirected URL here: ").strip()

    try:
        auth_code = raw_code.split("auth_code=")[1].split("&")[0]
    except IndexError:
        raise ValueError("Invalid redirect URL. Auth code not found.")

    # -------------------------------
    # Step 3: Exchange auth_code for access_token
    # -------------------------------
    session = fyersModel.SessionModel(
        client_id=client_id,
        secret_key=secret_key,
        redirect_uri=redirect_uri,
        response_type="code",
        grant_type="authorization_code"
    )

    session.set_token(auth_code)
    token_response = session.generate_token()

    return token_response


In [ ]:
client_id="ALT6RUE1IF-100"
secret_key="0UT0LW5PE4"
redirect_uri="https://luvratan.tech/"

token_response = generate_fyers_access_token(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri
)

access_token = token_response["access_token"]
print("Access Token:", access_token)


In [ ]:
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

data = {
    "symbol":"NSE:SBIN-EQ",
    "resolution":"15",
    "date_format":"1",
    "range_from":"2018-01-01",
    "range_to":"2018-03-31",
    "cont_flag":"1"
}

response = fyers.history(data=data)
print(response)



#### Establish Web socket connection

In [ ]:
from fyers_apiv3.FyersWebsocket import data_ws


def onerror(message):
    """
    Callback function to handle WebSocket errors.

    Parameters:
        message (dict): The error message received from the WebSocket.


    """
    print("Error:", message)


def onclose(message):
    """
    Callback function to handle WebSocket connection close events.
    """
    print("Connection closed:", message)


def onopen():
    """
    Callback function to subscribe to data type and symbols upon WebSocket connection.

    """
    # Specify the data type and symbols you want to subscribe to
    data_type = "SymbolUpdate"

    # Subscribe to the specified symbols and data type
    symbols = ['NSE:ADANIPORTS-EQ', 'NSE:SBIN-EQ']
    fyers.subscribe(symbols=symbols, data_type=data_type)

    # Keep the socket running to receive real-time data
    fyers.keep_running()




#### Check entry and exit triggers

In [ ]:
def check_trigger(ltp, resistance, support):
    if ltp <= support:
        print("Long Entry Triggered")
    elif ltp >= resistance:
        print("Short Entry Triggered")

In [ ]:
def onmessage(message):
    """
    Callback function to handle incoming messages from the FyersDataSocket WebSocket.

    Parameters:
        message (dict): The received message from the WebSocket.

    """
    # check_trigger(message["ltp"], 1384.20, 1370.20)
    print(f"Symbol: {message["symbol"]}", "LTP: ", message["ltp"])
    # print(message)

In [ ]:
# Create a FyersDataSocket instance with the provided parameters
fyers = data_ws.FyersDataSocket(
    access_token=access_token,       # Access token in the format "appid:accesstoken"
    log_path="",                     # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=False,                  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,              # Save response in a log file instead of printing it.
    reconnect=True,                  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,               # Callback function to subscribe to data upon connection.
    on_close=onclose,                # Callback function to handle WebSocket connection close events.
    on_error=onerror,                # Callback function to handle WebSocket errors.
    on_message=onmessage             # Callback function to handle incoming messages from the WebSocket.
)

# Establish a connection to the Fyers WebSocket
fyers.connect()


In [ ]:
# fyers.close_connection()
